## Question 1 ##
I start by resizing the image to the half of the previous size.
Then I continue by blurring to make it more consistent and have less amount of noise in the image.
I used canny edge detection method to store the edges in the edge array. In order to find the equivalent coefficients in hough space,
I used the range [0, 360) with steps of 2 degree for theta and also an array in size of the diagonal diameter in image for rho.
<br>
Then I continued by calculating rho for each white pixel(which indicates that it is on a line) and each theta. This results in all lines passing though this point and when I find this line, I increment the value in the index of [rho, theta] in the Accumulator.
If an index has a high value, we can conclude that there are reasonable amount of points on it and hence it is a line.
Next I set a threshold to find all of these lines. however, some of the lines are somehow the same or they do not belong to the chessboard.
So the next step was to remove these lines.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


def detectLines(img, i):
    img = cv2.resize(img, None, fx=0.5, fy=0.5)
    kernel = np.ones((3, 3), np.float32) / 3 ** 2
    blurred_img = cv2.filter2D(img, -1, kernel)
    edges = cv2.Canny(image=blurred_img, threshold1=350, threshold2=380)
    cv2.imwrite("../results/res0" + str(i) + ".jpg", edges)
    plt.imshow(edges)
    plt.show()

    height, width = edges.shape
    theta_size, rho_size = 180, int(np.ceil(np.sqrt(height ** 2 + width ** 2)))
    Acc = np.zeros((rho_size, theta_size))
    for (y, x) in list(zip(*np.where(edges == 255))):
        for t in range(0, theta_size):
            xt, yt = x, y
            cos = np.cos(np.deg2rad(2*t))
            sin = np.sin(np.deg2rad(2*t))
            rho = int(xt * cos + yt * sin)
            Acc[rho, t] += 1


    plt.imshow(img)
    cv2.imwrite("../results/res0" + str(i + 4) + "-lines.jpg", img)
    plt.show()

    threshold, cons = 50, 2
    (R, T) = np.where(Acc > threshold)
    C, S = np.cos(np.deg2rad(2 * T)), np.sin(np.deg2rad(2 * T))

    #separate lines:
    B, A = np.asarray(R / S, dtype=np.float16), np.asarray(-C / S, dtype=np.float16)



detectLines(cv2.imread('../images/im01.jpg'), 1)

Then I converted the hough space coefficients to get the slope a and b for x and y. I stored those values in arrays A and B.
in order to separate the lines from one another, I need to consider the values in B which are near each other and then select only one.
So I sort the arrays based on the args of sorted B and also sicne I want to compare the values in where the image is, I multiplied them by a x value in.
One thing we can do is to separate the ones with positive slope(a) and then we need to consider the B valued and then select them in a way that for each range,
the one smaller gets selected this removes the top line(redundant one). and Then similarly for the lines with neagative a.
another approach is to consider that each targer line, has black and white pixels around it.